In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from scipy.stats import randint, uniform

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [19]:
df = pd.read_csv('Данные для построения модели, задача - Классификация - превышает ли значение SI значение 8.csv')
df

,EState_VSA4,VSA_EState1,HallKierAlpha,VSA_EState7,SlogP_VSA5,SMR_VSA5,FractionCSP3,AvgIpc,BCUT2D_LOGPHI,SMR_VSA7,...,TPSA,VSA_EState4,EState_VSA8,SMR_VSA1,EState_VSA10,MinPartialCharge,MaxAbsEStateIndex,PEOE_VSA6,BCUT2D_MRLOW,Target Feature
0,24.925325,0.000000,-0.66,13.258223,105.750639,105.750639,0.923077,3.150503,2.644698,0.000000,...,24.72,4.807589,41.542423,0.000000,0.000000,-0.293526,5.094096,54.384066,0.193481,1
1,23.919494,0.000000,-0.08,14.135381,105.750639,117.834321,1.000000,3.150503,2.658342,0.000000,...,24.06,2.153503,52.176000,0.000000,0.000000,-0.313407,3.961417,54.384066,0.120322,0
2,23.919494,2.517630,-0.08,14.491619,105.750639,117.834321,1.000000,3.214947,2.679014,0.000000,...,0.00,2.184127,69.733111,8.966062,0.000000,-0.325573,2.627117,41.542423,-0.922902,0
3,24.925325,0.000000,-0.66,14.695439,112.171461,112.171461,0.925926,3.179270,2.644709,0.000000,...,24.72,4.827852,41.542423,0.000000,0.000000,-0.293526,5.097360,60.804888,0.193510,1
4,29.631406,0.000000,-2.22,8.627311,91.194256,86.488175,0.575758,3.337074,2.648473,59.657840,...,24.72,9.071783,90.073360,0.000000,0.000000,-0.257239,5.150510,65.807891,0.193687,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,38.524930,10.503509,-1.65,7.985276,66.219879,66.219879,0.800000,3.023764,2.739076,11.649125,...,69.67,0.498752,33.770969,23.857337,14.383612,-0.468587,12.934891,38.841158,-0.196385,0
997,38.524930,10.086396,-2.18,7.521836,73.143616,79.185457,0.785714,3.130958,2.738755,11.649125,...,98.77,0.162524,39.087758,28.651875,19.178149,-0.467493,13.635345,45.764895,-0.187625,0
998,44.277783,10.305031,-1.83,10.055493,72.640700,78.682541,0.800000,3.204255,2.738943,11.649125,...,98.77,0.139799,39.087758,28.651875,19.178149,-0.467485,13.991690,45.764895,-0.187687,0
999,6.420822,20.885832,-2.45,2.955837,66.219879,66.219879,0.821429,2.887043,2.704027,0.000000,...,122.27,-2.785593,6.923737,42.920138,23.972686,-0.468755,13.830180,27.192033,-0.225309,0


In [20]:
X, y = df.drop(['Target Feature'], axis=1), df['Target Feature']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [22]:
y.value_counts()

Target Feature
0    644
1    357
Name: count, dtype: int64

Классы не сбалансированны, произведем процесс балансировки классов

In [23]:
smote = SMOTE(sampling_strategy='auto', k_neighbors=10, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [24]:
X, y = X_resampled, y_resampled
y.value_counts()

Target Feature
1    644
0    644
Name: count, dtype: int64

Классы сбалансированны, можно приступать обучения моделей

In [25]:
class BestClassifierModel:
    """
    Класс для отбора лучшей модели машинного обучения для модели классификации
    X: DataFrame с признаками
    y: Series с целевой переменной

    В данном классе идет подбор 4 разных моделей машинного обучения.
    Для каждой модели производиться подор гипперпараметров (если таковые имеются)
    После получения моделей с лучшими параметрами идет отбор лучшей модели по средству сравнения assessment_criterion с полученными метриками моделей
    """
    def __init__(self, X, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        self.assessment_criterion = {
            'metric_name': 'F1-score',
            'value': 0.7
        }
        self.best_model = None
        self.models = {
            'linear_regression': {
                'model': LogisticRegression(max_iter=1000, random_state=42),
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': [0.01, 0.1, 1, 10],
                    'penalty': ['l1', 'l2'],
                    'solver': ['saga']
                },
                'parameter_selection_method': 'GridSearchCV',
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'elastic_net': {
                'model': KNeighborsClassifier(),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'GridSearchCV',
                'params': {
                    'n_neighbors': range(3, 15),
                    'weights': ['uniform', 'distance'],
                    'p': [1, 2]  # 1 = манхэттенское расстояние, 2 = евклидово расстояние
                },
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'random_forest_regressor': {
                'model': RandomForestClassifier(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'RandomizedSearchCV',
                'params': {
                    'n_estimators': randint(50, 200),
                    'max_depth': [None, 5, 10, 20],
                    'min_samples_split': randint(2, 10),
                    'min_samples_leaf': randint(1, 5)
                },
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': ''
                },
                'standartization': False,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'randomized_search_cv_model': {
                'model': SVC(random_state=42),
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': uniform(0.1, 10),
                    'kernel': ['linear', 'rbf'],
                    'gamma': ['scale', 'auto']
                },
                'parameter_selection_method': 'RandomizedSearchCV',
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                    'y_pred': None
                }
            }
        }
        
    
    def fit(self):
        """
        Метод подбора гипперпараметров и инизиализации лучших моделей с подобранными гипперпараметрами для каждой модели
        """
        for model in self.models:
            print(f"Подбор гипперпараметров для модели {str(self.models[model]['model'])} методом {self.models[model]['parameter_selection_method']}")
            
            X, y = self.models[model]['data']['X_train'], self.models[model]['data']['y_train']
            
            if self.models[model]['standartization']:
                print('Перед подобром, необходом провести стандартизацию параметров')
                X = self.X_train_stnd = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаем к подбору гиппепараметров')
            print('Ожидайте...')
            
            if self.models[model]['parameter_selection_method'] == 'GridSearchCV':
                grid = GridSearchCV(self.models[model]['model'], self.models[model]['params'], cv=5, scoring='neg_mean_squared_error')
                grid.fit(X, y)
                self.models[model]['best_params'] = grid.best_params_
                self.models[model]['best_model'] = grid.best_estimator_
            elif self.models[model]['parameter_selection_method'] == 'RandomizedSearchCV':
                search = RandomizedSearchCV(self.models[model]['model'], self.models[model]['params'], n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42)
                search.fit(X, y)
                self.models[model]['best_params'] = search.best_params_
                self.models[model]['best_model'] = search.best_estimator_
            print(f"Лучшие параметры для модели {str(self.models[model]['model'])}:", self.models[model]['best_params'])
            print()
                
    def get_best_model(self):
        """
        Метод получения лучшей модели по показателю assessment_criterion
        В данном методе происходит обучение моделей на тестовой выборке и получение метрик на основе полученных предсказаний
        
        По итогу работы модели мы получаем модель, которая имеет лучший показатель по критрию заданному в assessment_criterion или
        уведомление о том, что модель не была выбрана. Тогда необходимо либо изменить метрику, либо изменить значение метрики
        """
        for model in self.models:
            print(f"Обучаем модель {str(self.models[model]['model'])} с лучшими гипперпараметрами для модели:")
            
            X, y = self.models[model]['data']['X_test'], self.models[model]['data']['y_test']
            
            if self.models[model]['standartization']:
                print('Перед обучением, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
            
            print('Приступаем к формированию предсказания на тестовой выборке')
            
            self.models[model]['data']['y_pred'] = self.models[model]['best_model'].predict(X)
            
            self.models[model]['metrics']['Accurac'] = accuracy_score(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['Precision'] = precision_score(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['Recall'] = recall_score(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['F1-score'] = f1_score(y, self.models[model]['data']['y_pred'])
            
            print(f"Метрики для модели {str(self.models[model]['model'])}:")
            print(f"Accuracy: {self.models[model]['metrics']['Accurac']:.4f}")
            print(f"Precision: {self.models[model]['metrics']['Precision']:.4f}")
            print(f"Recall: {self.models[model]['metrics']['Recall']:.4f}")
            print(f"F1-score: {self.models[model]['metrics']['F1-score']:.4f}")
            print()

            if self.models[model]['metrics'][self.assessment_criterion['metric_name']] > self.assessment_criterion['value']:
                self.assessment_criterion['value'] = self.models[model]['metrics'][self.assessment_criterion['metric_name']]
                self.best_model = self.models[model]
        
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована')
            print('Измените value или metric_name в assessment_criterion')
        else:
            print('Модель для формирования прогноза сформирована')
            print(f"Лучшая модель по главному критерию оценки {self.assessment_criterion['metric_name']}:")
            print(f"{str(self.best_model['best_model'])}:")
            print('Метрики')
            for k, v in self.best_model['metrics'].items():
                print(f'{k} {v:.4f}')
            
    
    def predict(self, data):
        """
        Метод для формирования прогноза с помощью лучшей модели, которая была получена в методе get_best_model
        """
        result = None
        X = data
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована, используйте метод get_best_model')
            return result
        else:
            print(f"Формируем предсказание с помощью модели {str(self.best_model['best_model'])}")
            if self.best_model['standartization']:
                print('Перед формированияем прогноза, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(data) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаю формированию прогноза...')
            result = self.best_model['best_model'].predict(X)
            print('Предсказание сформировано')
        return result

In [26]:
best_regression_model = BestClassifierModel(X, y)
best_regression_model.fit()

Подбор гипперпараметров для модели LogisticRegression(max_iter=1000, random_state=42) методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...


g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_F

Лучшие параметры для модели LogisticRegression(max_iter=1000, random_state=42): {'C': 10, 'penalty': 'l1', 'solver': 'saga'}

Подбор гипперпараметров для модели KNeighborsClassifier() методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели KNeighborsClassifier(): {'n_neighbors': 4, 'p': 2, 'weights': 'distance'}

Подбор гипперпараметров для модели RandomForestClassifier(random_state=42) методом RandomizedSearchCV
Ожидайте...
Лучшие параметры для модели RandomForestClassifier(random_state=42): {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 141}

Подбор гипперпараметров для модели SVC(random_state=42) методом RandomizedSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели SVC

In [27]:
best_regression_model.get_best_model()

Обучаем модель LogisticRegression(max_iter=1000, random_state=42) с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели LogisticRegression(max_iter=1000, random_state=42):
Accuracy: 0.9354
Precision: 0.8971
Recall: 0.9786
F1-score: 0.9361

Обучаем модель KNeighborsClassifier() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели KNeighborsClassifier():
Accuracy: 0.7416
Precision: 0.7062
Recall: 0.7968
F1-score: 0.7487

Обучаем модель RandomForestClassifier(random_state=42) с лучшими гипперпараметрами для модели:
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели RandomForestClassifier(random_state=42):
Accuracy: 0.9121
Precisi

In [28]:
best_regression_model.predict(X_test)

Формируем предсказание с помощью модели LogisticRegression(C=10, max_iter=1000, penalty='l1', random_state=42,
                   solver='saga')
Перед формированияем прогноза, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаю формированию прогноза...
Предсказание сформировано


array([0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0]